# Overall Performance of Detecting, Transitioning and Aggression Lunging Simulation

In [1]:
import cv2
import numpy as np
from tqdm import trange
from flygym import Simulation, Camera
from flygym.arena import FlatTerrain

### For the different role of the flies, Class `LungingFly` and Class `WalkingFly` has been created 

In [2]:
from flies import LungingFly, WalkingFly

### First, remove parts of the fly's body that are in its field of vision

In [ ]:
# Remove parts of the fly's body that are in its field of vision
WalkingFly.config["vision"]["hidden_segments"] += [
    "LFFemur",
    "LFTibia",
    "LFTarsus1",
    "LFTarsus2",
    "LFTarsus3",
    "LFTarsus4",
    "LFTarsus5",
    "RFFemur",
    "RFTibia",
    "RFTarsus1",
    "RFTarsus2",
    "RFTarsus3",
    "RFTarsus4",
    "RFTarsus5",
    "LMFemur",
    "LMTibia",
    "LMTarsus1",
    "LMTarsus2",
    "LMTarsus3",
    "LMTarsus4",
    "LMTarsus5",
    "RMFemur",
    "RMTibia",
    "RMTarsus1",
    "RMTarsus2",
    "RMTarsus3",
    "RMTarsus4",
    "RMTarsus5",
]

### Manually add desired collison pairs using the function. It can be freely adjusted:

In [9]:
def add_collision_pair(fly1, fly2, geom_name1, geom_name2):
    geom1 = fly1.model.find("geom", geom_name1)
    geom2 = fly2.model.find("geom", geom_name2)
    fly1.model.contact.add(
        "pair",
        name=f"{geom_name1}_{geom_name2}",
        geom1=geom1,
        geom2=geom2,
        solref=fly1.contact_solref,
        solimp=fly1.contact_solimp,
    )

def add_all_collision_pairs(fly1, fly2):
    add_collision_pair(fly1, fly2, "Head","Thorax")
    add_collision_pair(fly1, fly2, "LFTarsus5","Head")
    add_collision_pair(fly1, fly2, "RFTarsus5","Head")
    add_collision_pair(fly1, fly2, "LFTarsus5","Thorax")
    add_collision_pair(fly1, fly2, "RFTarsus5","Thorax")
    add_collision_pair(fly1, fly2, "LFTarsus5","A1A2")
    add_collision_pair(fly1, fly2, "RFTarsus5","A1A2")
    add_collision_pair(fly1, fly2, "LFTarsus5","RWing")
    add_collision_pair(fly1, fly2, "RFTarsus5","RWing")
    add_collision_pair(fly1, fly2, "LFTarsus5","LWing")
    add_collision_pair(fly1, fly2, "RFTarsus5","LWing")


### To start, set up the time scale and initialize `the flies`, `the arena` and `the camera`


In [5]:
# set the running time
timestep = 1e-4
run_time = 0.5
t = np.arange(0, run_time, timestep)
average_pixels = []
# Initialize the 2 flies
lunging_fly = LungingFly(
    name="lunging",
    timestep=timestep,
    enable_adhesion=True,
    head_stabilization_model="thorax",
    neck_kp=1000,
    actuator_kp=30,    # Play with the actuator_kp to get different results！
    adhesion_force=40,
)

walking_fly = WalkingFly(
    name="walking",
    timestep=timestep,
    enable_adhesion=True,
    spawn_pos=(7, 3.55, 0.5),   # Play with the spawn_pos to get different results！
    spawn_orientation=(0, 0, 2*np.pi),
)

add_all_collision_pairs(lunging_fly, walking_fly)   # Enabling the collision

arena = FlatTerrain(ground_alpha=0) # Remove the background color 

#Multi-camera Simulation
birdeye_cam_zoom = arena.root_element.worldbody.add(
    "camera",
    name="birdeye_cam_zoom",
    mode="fixed",
    pos=(15, 0, 20),
    euler=(0, 0, 0),
    fovy=45,
)

birdeye_cam = arena.root_element.worldbody.add(
    "camera",
    name="birdeye_cam",
    mode="fixed",
    pos=(15, 0, 35),
    euler=(0, 0, 0),
    fovy=45,
)

cam = Camera(
    fly=lunging_fly,
    camera_id="Animat/camera_left", #Play with the camera_id (such as "Animat/camera_top") to get different results！
    play_speed=0.2,
    window_size=(800, 608),
)

sim = Simulation(
    flies=[lunging_fly, walking_fly],
    cameras=[cam],
    arena=arena,
    timestep=timestep,
)

In [6]:
obs, info = sim.reset(seed=0)
obs["lunging"], info_luning = obs["lunging"], info["lunging"]
second_cam_frames = []
x = None
alpha = 1e-1  #play with alpha calue to get different chasing results

In [7]:
stopping_steps = 0 #Time to make the lunging fly to reset its move  

# Warning:Since the action of np.zero() is Not the ready-to-lunge action,the stopping_step has been set to 0.
# The transition action has been moved to the flies.py, in the datablock of the joint angles of lunging action.

for i in trange(t):
    visual_features  = lunging_fly.process_visual_observation(obs["lunging"]["vision"])
    averages = lunging_fly.average_pixels(obs["lunging"]["vision"])

    left_deviation = 1 - visual_features[1]
    right_deviation = visual_features[4]

    left_found = visual_features[2] > 0.001
    right_found = visual_features[5] > 0.001  #play with the value to set different threshold！
 
    left_detected = averages[0] < 0.485
    right_detected = averages[1] < 0.485  #play with the value to set different threshold!
    
    if i % 500 == 0:   #every 0.05s, print the average visual input feature to keep track of the fly
        print(averages)
        print(f"left:{left_detected},right:{right_detected}" )
        print(visual_features[2])
        print(visual_features[5])
        average_pixels.append(averages)
    if left_detected + right_detected == 0:
        lunging_fly.detected = False
        if not left_found:
            left_deviation = np.nan

        if not right_found:
            right_deviation = np.nan

        lunging_action_chasing = np.array(
            [
                lunging_fly.calc_ipsilateral_speed(left_deviation, left_found),
                lunging_fly.calc_ipsilateral_speed(right_deviation, right_found),
                0,
            ]
        )

        obs, reward, terminated, truncated, info = sim.step({
            "lunging": lunging_action_chasing ,
            "walking": 0.6*np.ones(2),     #Play with the walking fly's walking speed to get different detecting results
        })

        render_res = sim.render()[0]

        if render_res is not None:
            lunging_fly.visual_inputs_hist.append(obs["lunging"]["vision"].copy())
            second_cam = sim.physics.bind(birdeye_cam_zoom)
            x_new = sim._get_center_of_mass()[0]

            if x is None:
                x = x_new

            x = (1 - alpha) * x + alpha * x_new

            second_cam.pos[0] = x
            second_img = sim.physics.render(
                width=700, height=560, camera_id="birdeye_cam_zoom"
            )
            second_img = cv2.putText(
                np.ascontiguousarray(second_img),
                f"{sim.cameras[0].play_speed}x",
                org=(20, 30),
                fontFace=cv2.FONT_HERSHEY_DUPLEX,
                fontScale=0.8,
                color=(0, 0, 0),
                lineType=cv2.LINE_AA, 
                thickness=1,
            )
            second_cam_frames.append(second_img)
    else:
        break
print("Dectected!")
for i in range(stopping_steps + lunging_fly.data.shape[1]):
    if i < stopping_steps:
        lunging_action = np.zeros(3)
    else:
        lunging_action = np.array([0, 0, 1])    #start lunging!
    obs, reward, terminated, truncated, info = sim.step({
        "lunging": lunging_action,
        "walking": 0.6*np.ones(2),  #play with the walking fly's walking speed to get different detecting results!
    })

    render_res = sim.render()[0]

    if render_res is not None:
        lunging_fly.visual_inputs_hist.append(obs["lunging"]["vision"].copy()) # Keep track of the lunging fly's visual input to make a video


  0%|          | 21/5000 [00:00<00:43, 113.60it/s]

[0.49755263 0.49998498]
left:False,right:False
0.0
0.0


 10%|█         | 509/5000 [00:05<00:53, 83.61it/s] 

[0.49744034 0.49991086]
left:False,right:False
0.0
0.0


 20%|██        | 1009/5000 [00:11<00:48, 81.71it/s]

...

[0.48710668 0.49985585]
left:False,right:False
0.0027739252
0.0


 90%|█████████ | 4508/5000 [01:00<00:07, 63.26it/s]

[0.48549497 0.49988002]
left:False,right:False
0.008321775
0.0


 90%|█████████ | 4520/5000 [01:00<00:06, 74.74it/s]


Dectected!


### Plot the average weighted pixel value over time

In [ ]:
import matplotlib.pyplot as plt
x = list(range(len(lunging_fly.visual_inputs_hist)))
y1 = [point[0] for point in average_pixels]  
y2 = [point[1] for point in average_pixels]  
plt.plot( x,y1, label='average_pixels_left')
plt.plot( x,y2, label='average_pixels_right')
plt.title('Average Pixels Values Over Time')
plt.xlabel('t/0.01s')
plt.ylabel('Average Pixels Values')
plt.legend()
plt.show()

### Render the video


In [8]:
from flygym.vision import save_video_with_vision_insets

sim.fly = lunging_fly

save_video_with_vision_insets(
    sim,
    cam,
    "outputs/lunging.mp4",
    lunging_fly.visual_inputs_hist,
)

### Play the video

In [138]:
from IPython.display import Video

Video("outputs/lunging.mp4")